# Project 5: Logic Networks

## NAND Gate

In [1]:
from vpython import *
import math
import numpy as np
import matplotlib.pyplot as plt
import heapq, time

<IPython.core.display.Javascript object>

In [2]:
class Gate:
    _id_counter = 0

    def __init__(self, input_nets, output_net, logic_fn, delay=7, name=None):
        self.id = Gate._id_counter
        Gate._id_counter += 1

        self.name = name if name else f"Gate_{self.id}"
        self.inputs = input_nets
        self.output = output_net
        self.logic_fn = logic_fn
        self.delay = delay

        if len(self.inputs) < 2:
            print("gates must have two or more inputs")

        for net in self.inputs:
            net.add_receiver(self)
        self.output.set_driver(self)

    def evaluate(self, time, simulator):
        input_values = [net.value for net in self.inputs]
        result = self.logic_fn(*input_values)
        self.output.set(result, time + self.delay, simulator)


In [3]:
class Net:
    _id_counter = 0

    def __init__(self, name=None):
        self.id = Net._id_counter
        Net._id_counter += 1

        self.name = name if name else f"net_{self.id}"
        self.value = 'X'
        self.driver = None
        self.receivers = []

    def set_driver(self, gate):
        if self.driver:
            print("Net already has a driver")
        self.driver = gate

    def add_receiver(self, gate):
        self.receivers.append(gate)

    def set(self, value, time, simulator):
        if self.value != value:
            simulator.schedule_event(Event(self, value, time))


In [4]:
class Event:
    def __init__(self, net, value, time):
        self.net_id = net.id
        self.net = net
        self.value = value
        self.time = time

    # Define less than comparison for sorting events by time
    def __lt__(self, other):
        return self.time < other.time

    # Optional: For debugging purposes, you can add a string representation
    def __repr__(self):
        return f"Event(net={self.net.name}, value={self.value}, time={self.time})"

In [5]:
class EventQueue:
    def __init__(self):
        self.events = []

    def post_event(self, event):
        # Push the event into the heap (priority queue)
        heapq.heappush(self.events, event)

    def next_event(self):
        # Pop the next event with the earliest time
        return heapq.heappop(self.events) if self.events else None

    def has_events(self):
        # Check if there are any events left in the queue
        return len(self.events) > 0

In [6]:
class Simulator:
    def __init__(self):
        self.time = 0
        self.event_queue = EventQueue()

    def schedule_event(self, event):
        self.event_queue.post_event(event)

    def run(self):
        print("=== Starting Simulation ===")
        while self.event_queue.has_events():
            event = self.event_queue.next_event()
            self.time = event.time
            affected_gates = set()

            net = event.net
            print(f"[{self.time}ns] Net '{net.name}' (ID {net.id}) = {event.value}")
            if net.value != event.value:
                #print(f"[{self.time}ns] Net '{net.name}' (ID {net.id}) = {event.value}")
                net.value = event.value
                affected_gates.update(net.receivers)    
            for gate in affected_gates:
                gate.evaluate(self.time, self)
            update_graph()
            rate(10)
        print("=== Simulation Complete ===")

In [7]:
def nand(input1, input2):
    if input1 == 'X' or input2 == 'X':
        return 'X'
    return 1 if not (input1 and input2) else 0

def nor(input1, input2):
    if input1 == 'X' or input2 == 'X':
        return 'X'
    return 1 if not (input1 or input2) else 0

In [8]:
def update_graph():
    if A.value != 'X':
        A_curve.plot(sim.time, int(A.value))  # Convert to int if needed
    if B.value != 'X':
        B_curve.plot(sim.time, int(B.value))
    if C.value != 'X':
        C_curve.plot(sim.time, int(C.value))

In [9]:
scene = canvas()
graph_scene = graph(title="Signal Graph", xtitle="Time (ns)", ytitle="Value")
A_curve = gcurve(color=color.red, label="A",visible=True)
B_curve = gcurve(color=color.green, label="B",visible=True)
C_curve = gcurve(color=color.blue, label="C",visible=True)

sim = Simulator()

# Define nets
A = Net("A")
B = Net("B")
C = Net("C")  # Output of G1

# Define gates
G1 = Gate([A, B], C, nand, delay=7, name="NAND1")

# A B NAND
# 0 0 1
# 0 1 1
# 1 0 1
# 1 1 0

# Set input values at time 0
A.set(0, 0, sim)
B.set(0, 0, sim)

A.set(0, 10, sim)
B.set(1, 10, sim)

A.set(1, 20, sim)
B.set(0, 20, sim)

A.set(1, 30, sim)
B.set(1, 30, sim)

# Run simulation
sim.run()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

=== Starting Simulation ===
[0ns] Net 'A' (ID 0) = 0
[0ns] Net 'B' (ID 1) = 0
[7ns] Net 'C' (ID 2) = 1
[10ns] Net 'A' (ID 0) = 0
[10ns] Net 'B' (ID 1) = 1
[20ns] Net 'B' (ID 1) = 0
[20ns] Net 'A' (ID 0) = 1
[30ns] Net 'B' (ID 1) = 1
[30ns] Net 'A' (ID 0) = 1
[37ns] Net 'C' (ID 2) = 0
=== Simulation Complete ===


### Results

The logic gate's value does not get added to the queue unless it changes. Using the truth table, we can see that the NAND gate will be 1 for all A and B except for when A and B are 1 and 1. Therefore, Net C first changes to 1 at 7ns after the delay, and does not change until 0 at 37ns.

We can see that using all possibl combinations for the NAND gate input we get the proper outputs.

### Assumptions

7 nanosecond delay in a NAND gate: https://pages.hep.wisc.edu/~prepost/623/digital1_2.html#:~:text=Level%20~%201.4%20V.-,The%20delay%20in%20passing%20a%20signal%20through%20one%20NAND%20gate,speed%20and%20input%20power%20required.

In [10]:
def update_graph():
    if A.value != 'X':
        A_curve.plot(sim.time, int(A.value))  # Convert to int if needed
    if B.value != 'X':
        B_curve.plot(sim.time, int(B.value))
    if C.value != 'X':
        C_curve.plot(sim.time, int(C.value))
    if D.value != 'X':
        D_curve.plot(sim.time, int(D.value))
    if OUT.value != 'X':
        OUT_curve.plot(sim.time, int(OUT.value))

In [11]:
scene = canvas()
graph_scene = graph(title="Signal Graph", xtitle="Time (ns)", ytitle="Value")
A_curve = gcurve(color=color.red, label="A",visible=True)
B_curve = gcurve(color=color.green, label="B",visible=True)
C_curve = gcurve(color=color.blue, label="C",visible=True)
D_curve = gcurve(color=color.orange, label="D",visible=True)
OUT_curve = gcurve(color=color.magenta, label="OUT",visible=True)

sim = Simulator()

A = Net("A")
B = Net("B")
C = Net("C")     # G1 output
D = Net("D")     # G2 output
OUT = Net("OUT") # G3 output (E)

G1 = Gate([A, B], C, nand, delay=7, name="G1")
G2 = Gate([A, C], D, nand, delay=7, name="G2")
G3 = Gate([B, D], OUT, nand, delay=7, name="G3")

# A=0, B=0 → expect C=1, D=1, OUT=1
# A=0, B=1 → expect C=1, D=1, OUT=0
# A=1, B=0 → expect C=1, D=0, OUT=1
# A=1, B=1 → expect C=0, D=1, OUT=0

A.set(0, 0, sim)
B.set(0, 0, sim)

A.set(0, 15, sim)
B.set(1, 15, sim)

A.set(1, 30, sim)
B.set(0, 30, sim)

A.set(1, 45, sim)
B.set(1, 45, sim)

sim.run()

<IPython.core.display.Javascript object>

=== Starting Simulation ===
[0ns] Net 'A' (ID 3) = 0
[0ns] Net 'B' (ID 4) = 0
[7ns] Net 'C' (ID 5) = 1
[14ns] Net 'D' (ID 6) = 1
[15ns] Net 'A' (ID 3) = 0
[15ns] Net 'B' (ID 4) = 1
[21ns] Net 'OUT' (ID 7) = 1
[22ns] Net 'OUT' (ID 7) = 0
[30ns] Net 'A' (ID 3) = 1
[30ns] Net 'B' (ID 4) = 0
[37ns] Net 'D' (ID 6) = 0
[37ns] Net 'C' (ID 5) = 0
[37ns] Net 'OUT' (ID 7) = 1
[44ns] Net 'OUT' (ID 7) = 1
[44ns] Net 'D' (ID 6) = 1
[45ns] Net 'B' (ID 4) = 1
[45ns] Net 'A' (ID 3) = 1
[52ns] Net 'OUT' (ID 7) = 0
=== Simulation Complete ===


### Results

Going through the simulation step by step shows that it is accurate. For example, we start with A and B being 0. This should result in C being 1, D being 1, and E being 1. The delay of one NAND gate is 7 nanoseconds. At 7 nanoseconds, the first NAND gate (C) returns 1. After another 7, the D NAND gate returns 1 as well. After another 7, at 21 nanoseconds, the final gate E/OUT returns 1 as well. The delays in the NAND gates affect how the outputs of the NAND gates appear in the graph and list.

The second pair of A and B being 0 and 1 comes before the first circuit has fully finished. The change in B value triggers the OUT gate to be changed to 0 as you can see at 22 nanoseconds, 7 nanoseconds after the B net is changed at 15 nanoseconds. A change in C and D is not seen in the graph or list because their values remain.

Going through the rest of the list continuously verifies the results

## NOR Gate

In [12]:
def update_graph():
    if A.value != 'X':
        A_curve.plot(sim.time, int(A.value))  # Convert to int if needed
    if B.value != 'X':
        B_curve.plot(sim.time, int(B.value))
    if C.value != 'X':
        C_curve.plot(sim.time, int(C.value))

In [14]:
scene = canvas()
graph_scene = graph(title="Signal Graph", xtitle="Time (ns)", ytitle="Value")
A_curve = gcurve(color=color.red, label="A",visible=True)
B_curve = gcurve(color=color.green, label="B",visible=True)
C_curve = gcurve(color=color.blue, label="C",visible=True)

sim = Simulator()

# Define nets
A = Net("A")
B = Net("B")
C = Net("C")  # Output of G1

# Define gates
G1 = Gate([A, B], C, nor, delay=15, name="nor1")

# A B NAND
# 0 0 1
# 0 1 0
# 1 0 0
# 1 1 0

# Set input values at time 0
A.set(0, 0, sim)
B.set(0, 0, sim)

A.set(0, 10, sim)
B.set(1, 10, sim)

A.set(1, 20, sim)
B.set(0, 20, sim)

A.set(1, 30, sim)
B.set(1, 30, sim)

# Run simulation
sim.run()

<IPython.core.display.Javascript object>

=== Starting Simulation ===
[0ns] Net 'A' (ID 11) = 0
[0ns] Net 'B' (ID 12) = 0
[10ns] Net 'A' (ID 11) = 0
[10ns] Net 'B' (ID 12) = 1
[15ns] Net 'C' (ID 13) = 1
[20ns] Net 'A' (ID 11) = 1
[20ns] Net 'B' (ID 12) = 0
[25ns] Net 'C' (ID 13) = 0
[30ns] Net 'B' (ID 12) = 1
[30ns] Net 'A' (ID 11) = 1
[35ns] Net 'C' (ID 13) = 0
[35ns] Net 'C' (ID 13) = 0
=== Simulation Complete ===


### Results

Inputting A and B as 0 into a NOR gate will return 1. After 15 nanoseconds, the C gate becomes 1. The NOR logic follows for the rest of the inputs, as the rest of the C gates should be 0.

### Assumptions

15 nanosecond propagation delay: https://www.futurlec.com/74/IC7402.shtml

In [15]:
def update_graph():
    if A.value != 'X':
        A_curve.plot(sim.time, int(A.value))  # Convert to int if needed
    if B.value != 'X':
        B_curve.plot(sim.time, int(B.value))
    if C.value != 'X':
        C_curve.plot(sim.time, int(C.value))
    if D.value != 'X':
        D_curve.plot(sim.time, int(D.value))
    if OUT.value != 'X':
        OUT_curve.plot(sim.time, int(OUT.value))

In [17]:
scene = canvas()
graph_scene = graph(title="Signal Graph", xtitle="Time (ns)", ytitle="Value")
A_curve = gcurve(color=color.red, label="A",visible=True)
B_curve = gcurve(color=color.green, label="B",visible=True)
C_curve = gcurve(color=color.blue, label="C",visible=True)
D_curve = gcurve(color=color.orange, label="D",visible=True)
OUT_curve = gcurve(color=color.magenta, label="OUT",visible=True)

sim = Simulator()

A = Net("A")
B = Net("B")
C = Net("C")     # G1 output
D = Net("D")     # G2 output
OUT = Net("OUT") # G3 output (E)

G1 = Gate([A, B], C, nor, delay=15, name="G1")
G2 = Gate([A, C], D, nor, delay=15, name="G2")
G3 = Gate([B, D], OUT, nor, delay=15, name="G3")

# A=0, B=0 → expect C=1, D=0, OUT=1
# A=0, B=1 → expect C=0, D=1, OUT=0
# A=1, B=0 → expect C=0, D=0, OUT=1
# A=1, B=1 → expect C=0, D=0, OUT=0

A.set(0, 0, sim)
B.set(0, 0, sim)

A.set(0, 30, sim)
B.set(1, 30, sim)

A.set(1, 60, sim)
B.set(0, 60, sim)

A.set(1, 90, sim)
B.set(1, 90, sim)

sim.run()

<IPython.core.display.Javascript object>

=== Starting Simulation ===
[0ns] Net 'A' (ID 19) = 0
[0ns] Net 'B' (ID 20) = 0
[15ns] Net 'C' (ID 21) = 1
[30ns] Net 'A' (ID 19) = 0
[30ns] Net 'D' (ID 22) = 0
[30ns] Net 'B' (ID 20) = 1
[45ns] Net 'OUT' (ID 23) = 1
[45ns] Net 'OUT' (ID 23) = 0
[45ns] Net 'C' (ID 21) = 0
[60ns] Net 'B' (ID 20) = 0
[60ns] Net 'A' (ID 19) = 1
[60ns] Net 'D' (ID 22) = 1
[75ns] Net 'C' (ID 21) = 1
[75ns] Net 'OUT' (ID 23) = 1
[90ns] Net 'A' (ID 19) = 1
[90ns] Net 'B' (ID 20) = 1
[90ns] Net 'D' (ID 22) = 0
[105ns] Net 'OUT' (ID 23) = 0
[105ns] Net 'C' (ID 21) = 0
[105ns] Net 'OUT' (ID 23) = 0
=== Simulation Complete ===


### Results

Starting off with A and B being 0, C should be 1, D should be 0, and OUT should be 1. At 15 nanoseconds C becomes 1. At 30 nanoseconds D becomes 0, and at 45 nanoseconds OUT becomes 1. This logic follows correctly as each gate delay is 15 nanoseconds. At 30 nanoseconds, B becomes 1. This triggers OUT to become 0 at 45 nanoseconds. At the same time, C becomes 0. Following the order of events with the gate delays shows that the logic works correctly.


## NAND Gate D-Latch

In [25]:
def update_graph():
    if D.value != 'X':
        D_curve.plot(sim.time, int(D.value))  # Convert to int if needed
    if EN.value != 'X':
        EN_curve.plot(sim.time, int(EN.value))
    if Dbar.value != 'X':
        Dbar_curve.plot(sim.time, int(Dbar.value))
    if S.value != 'X':
        S_curve.plot(sim.time, int(S.value))
    if R.value != 'X':
        R_curve.plot(sim.time, int(R.value))
    if Q.value != 'X':
        Q_curve.plot(sim.time, int(Q.value))
    if Qbar.value != 'X':
        Qbar_curve.plot(sim.time, int(Qbar.value))

In [39]:
scene = canvas()
graph_scene = graph(title="Signal Graph", xtitle="Time (ns)", ytitle="Value")
Dbar_curve = gcurve(color=color.red, label="Dbar",visible=True)
D_curve = gcurve(color=color.green, label="D",visible=True)
S_curve = gcurve(color=color.blue, label="S",visible=True)
R_curve = gcurve(color=color.orange, label="R",visible=True)
Qbar_curve = gcurve(color=color.cyan, label="Qbar",visible=True)
Q_curve = gcurve(color=color.magenta, label="Q",visible=True)
EN_curve = gcurve(color=color.yellow, label="EN",visible=True)

sim = Simulator()

D = Net("D")
EN = Net("EN")
Dbar = Net("Dbar")
S = Net("S")
R = Net("R")
Q = Net("Q")
Qbar = Net("Qbar")

INV_D = Gate([D, D], Dbar, nand, delay=7, name="INV_D")
G1 = Gate([D, EN], S, nand, delay=7, name="G1")
G2 = Gate([D, EN], R, nand, delay=7, name="G2")
G3 = Gate([S, Qbar], Q, nand, delay=7, name="G3")
G4 = Gate([R, Q], Qbar, nand, delay=7, name="G4")


# Initial state: EN = 0 
Q.set(0,0,sim)
D.set(0, 0, sim)
EN.set(0, 0, sim)
Dbar.set(1,0,sim)

# Enable and set D = 1
EN.set(1, 10, sim)
D.set(1, 10, sim)

# Disable latch 
EN.set(0, 20, sim)

# Change D to 0 
D.set(0, 30, sim)

# Re-enable and set D = 0 
EN.set(1, 40, sim)

# Finish
EN.set(0, 50, sim)

sim.run()


<IPython.core.display.Javascript object>

=== Starting Simulation ===
[0ns] Net 'Q' (ID 147) = 0
[0ns] Net 'EN' (ID 143) = 0
[0ns] Net 'D' (ID 142) = 0
[0ns] Net 'Dbar' (ID 144) = 1
[7ns] Net 'Dbar' (ID 144) = 1
[7ns] Net 'R' (ID 146) = 1
[7ns] Net 'S' (ID 145) = 1
[10ns] Net 'D' (ID 142) = 1
[10ns] Net 'EN' (ID 143) = 1
[14ns] Net 'Qbar' (ID 148) = 1
[14ns] Net 'Q' (ID 147) = X
[17ns] Net 'Dbar' (ID 144) = 0
[17ns] Net 'S' (ID 145) = 0
[17ns] Net 'R' (ID 146) = 0
[20ns] Net 'EN' (ID 143) = 0
[21ns] Net 'Qbar' (ID 148) = X
[24ns] Net 'Q' (ID 147) = 1
[24ns] Net 'Qbar' (ID 148) = X
[27ns] Net 'S' (ID 145) = 1
[27ns] Net 'R' (ID 146) = 1
[30ns] Net 'D' (ID 142) = 0
[31ns] Net 'Qbar' (ID 148) = 1
[34ns] Net 'Qbar' (ID 148) = 0
[34ns] Net 'Q' (ID 147) = X
[37ns] Net 'Dbar' (ID 144) = 1
[38ns] Net 'Q' (ID 147) = 0
[40ns] Net 'EN' (ID 143) = 1
[41ns] Net 'Qbar' (ID 148) = X
[45ns] Net 'Qbar' (ID 148) = 1
[48ns] Net 'Q' (ID 147) = X
[50ns] Net 'EN' (ID 143) = 0
[55ns] Net 'Qbar' (ID 148) = X
=== Simulation Complete ===


### Results

At 0 nanoseconds, the enable is set to 0 which means that the latch is disabled and the state is held. Since D is 0, Dbar is 1.At 7 nanoseconds, R and S become 1. After another delay, at 14 nanoseconds Qbar becomes 1 which follows the logic correctly. At 10 nanoseconds, the enable is set to 1 meaning that the latch is now enabled. Since D is 1, Q and Qbar should be set to 1 and 0. Because of the NAND gate delays, these changes aren't seen until 24 nanoseconds. Tracking the nanoseconds and gate delays for the rest of the values verifies the logic of the NAND D-latch.

The simulation is stopped by setting the enable back to 0 at 50 nanoseconds.

## Combinational VS Sequential

Combinational circuits are far easier to track as their states show immediate results after any state changes. This also makes them far easier to debug because they are so easily traceable. Sequential circuits are more complicated to trace and since there are feedback loops there are a lot of undefined states/"X" which can be intimidated and make the circuit appear as if it is not working. This also makes it harder to debug. Any state changes in the circuit have large ripple affects that change the rest of the circuit.